# Import

In [1]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from smart_open import open
from tqdm import tqdm

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [3]:
from ALL import config
from util import *

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [7]:
data_types = ["20News", "AgNews", "AgNewsTitle"]

# Functions

In [6]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)

# Read data

In [10]:
for data_type in data_types:
    # データ読み込み
    df = pd.read_csv(f"../Preprocessing/data/{data_type}/master.csv", index_col=0)
    label = df["class"].to_numpy()
    # コーパスの作成
    texts = df.words_nonstop.progress_apply(lambda x: x.split(" ") if x is not np.nan else [""]).tolist()
    corpus = Corpus(texts=texts)
    dictionary = Dictionary(texts)
    dictionary.filter_extremes()

    # coherenceの計算
    cm = CoherenceModel(
        topics=label.reshape(1, -1),
        corpus=corpus,
        dictionary=dictionary,
        texts=texts,
        coherence="c_v",
    )
    coherence_path = f"data/{data_type}/coherence.csv"
    os.makedirs(os.path.dirname(coherence_path), exist_ok=True)
    pd.DataFrame([cm.get_coherence()]).to_csv(coherence_path)

100%|██████████| 120000/120000 [00:00<00:00, 389089.52it/s]
